## Naming conventions

The python community has some naming convections, defined in PEP-8:

https://www.python.org/dev/peps/pep-0008/

The widely adopted ones are:

* class names start with an uppercase, and use "camelcase" for multiword names, e.g. `ShoppingCart`

* varible names (including objects which are instances of a class) are lowercase and use underscores to separate words, e.g., `shopping_cart`

* module names should be lowercase with underscores



# Practicing Classes

## Exercise 1 (shopping cart)

Let's write a simple shopping cart class -- this will hold items that you intend to purchase as well as the amount, etc.  And allow you to add / remove items, get a subtotal, etc.

We'll use two classes: `Item` will be a single item and `ShoppingCart` will be the collection of items you wish to purchase.

First, our store needs an inventory -- here's what we have for sale:

In [143]:
INVENTORY_TEXT = """
apple, 0.60
banana, 0.20
grapefruit, 0.75
grapes, 1.99
kiwi, 0.50
lemon, 0.20
lime, 0.25
mango, 1.50
papaya, 2.95
pineapple, 3.50
blueberries, 1.99
blackberries, 2.50
peach, 0.50
plum, 0.33
clementine, 0.25
cantaloupe, 3.25
pear, 1.25
quince, 0.45
orange, 0.60
"""

# this will be a global -- convention is all caps
INVENTORY = {}
for line in INVENTORY_TEXT.splitlines():
    if line.strip() == "":
        continue
    item, price = line.split(",")
    INVENTORY[item] = float(price)


In [144]:
INVENTORY

{'apple': 0.6,
 'banana': 0.2,
 'grapefruit': 0.75,
 'grapes': 1.99,
 'kiwi': 0.5,
 'lemon': 0.2,
 'lime': 0.25,
 'mango': 1.5,
 'papaya': 2.95,
 'pineapple': 3.5,
 'blueberries': 1.99,
 'blackberries': 2.5,
 'peach': 0.5,
 'plum': 0.33,
 'clementine': 0.25,
 'cantaloupe': 3.25,
 'pear': 1.25,
 'quince': 0.45,
 'orange': 0.6}

### `Item` 

Here's the start of an item class -- we want it to hold the name and quantity.  

You should have the following features:

* the name should be something in our inventory

* Our shopping cart will include a list of all the items we want to buy, so we want to be able to check for duplicates.  Implement the equal test, `==`, using `__eq__`

* we'll want to consolidate dupes, so implement the `+` operator, using `__add__` so we can add items together in our shopping cart.  Note, add should raise a ValueError if you try to add two `Items` that don't have the same name.

Here's a start:

In [145]:
class Item(object):
    """ an item to buy """
    
    def __init__(self, name, quantity=1):
        if name not in INVENTORY:
            raise ValueError
        self.name = name
        self.quantity = quantity
        
    def __repr__(self):
        return "Item({},{})".format(self.name,self.quantity)
        
    def __eq__(self, other):
        return (self.name == other.name)
    
    def __add__(self, other):
        if self == other:
            return Item(self.name,self.quantity + other.quantity)
        else:
            raise ValueError

Here are some tests your code should pass:

In [146]:
a = Item("apple", 10)
b = Item("banana", 20)

In [147]:
c = Item("apple", 20)

In [148]:
# won't work
a + b

ValueError: 

In [149]:
# will work
a += c

In [150]:
a

Item(apple,30)

In [151]:
a == b

False

In [152]:
a == c

True

How do they behave in a list?

In [153]:
items = []
items.append(a)
items.append(b)
items

[Item(apple,30), Item(banana,20)]

In [154]:
c in items

True

### `ShoppingCart`

Now we want to create a shopping cart.  The main thing it will do is hold a list of items.

In [155]:
class ShoppingCart(object):
    
    def __init__(self):
        self.items = []
        
    def subtotal(self):
        """ return a subtotal of our items """
        st = []
        for item in self.items:
            st.append(item.quantity*INVENTORY[item.name])
        return sum(st)
        
    def add(self, name, quantity):
        """ add an item to our cart """
        if name not in INVENTORY:
            raise ValueError
        item_present = False
        for i in range(0,len(self.items)):
            try:
                self.items[i] += Item(name,quantity)
                item_present = True
            except:
                pass
        if not item_present:
            self.items.append(Item(name,quantity))
        
    def remove(self, name):
        """ remove all of item name from the cart """
        if Item(name,1) not in self.items:
            raise ValueError
        for i in range(0,len(self.items)):
            if self.items[i].name == name:
                self.items.pop(i)
        
    def report(self):
        """ print a summary of the cart """
        for item in self.items:
            print("{}: {}".format(item.name,item.quantity))
        

Here are some tests

In [156]:
sc = ShoppingCart()
sc.add("orange", 19)

In [157]:
sc.add("apple", 2)

In [158]:
sc.report()

orange: 19
apple: 2


In [159]:
sc.add("apple", 9)

In [160]:
sc.report()

orange: 19
apple: 11


In [161]:
sc.subtotal()

18.0

In [162]:
sc.remove("apple")

In [163]:
sc.report()

orange: 19


## Exercise 2: Poker Odds

Use the deck of cards class from the notebook we worked through outside of class to write a _Monte Carlo_ code that plays a lot of hands of straight poker (like 100,000).  Count how many of these hands has a particular poker hand (like 3-of-a-kind).  The ratio of # of hands with 3-of-a-kind to total hands is an approximation to the odds of getting a 3-of-a-kind in poker.

You'll want to copy-paste those classes into a `.py` file to allow you to import and reuse them here

In [164]:
# copy-pasting here so that a local .py file is not needed for others to run my code
# also, a change was made

class Card:
    
    def __init__(self, suit=1, rank=2):
        if suit < 1 or suit > 4:
            print("invalid suit, setting to 1")
            suit = 1
            
        self.suit = suit
        self.rank = rank
        

    def value(self):
        """ we want things order primarily by rank then suit """
        return self.suit + (self.rank-2)*4  #changed 14 to 4 and -1 to -2
    
    # we include this to allow for comparisons with < and > between cards 
    def __lt__(self, other):
        return self.value() < other.value()

    def __eq__(self, other):
        return self.value() == other.value()
    
    def __hash__(self):
        return hash((self.rank, self.suit))
    
    def __str__(self):
        suits = ["\u2660",  # spade
                 "\u2665",  # heart
                 "\u2666",  # diamond
                 "\u2663"]  # club
        
        r = str(self.rank)
        if self.rank == 11:
            r = "J"
        elif self.rank == 12:
            r = "Q"
        elif self.rank == 13:
            r = "K"
        elif self.rank == 14:
            r = "A"
                
        return r +':'+suits[self.suit-1]
        

In [165]:
import random

class Deck:
    """ the deck is a collection of cards """

    def __init__(self):

        self.nsuits = 4
        self.nranks = 13
        self.minrank = 2
        self.maxrank = self.minrank + self.nranks - 1

        self.cards = []

        for rank in range(self.minrank,self.maxrank+1):
            for suit in range(1, self.nsuits+1):
                self.cards.append(Card(rank=rank, suit=suit))

    def shuffle(self):
        random.shuffle(self.cards)

    def get_cards(self, num=1):
        hand = []
        for n in range(num):
            hand.append(self.cards.pop())

        return hand
    
    def __str__(self):
        string = ""
        for c in self.cards:
            string += str(c) + " "
        return string

In [170]:
def monte_carlo(num_hands): 
    types_of_hands = {"straight_flush":0,"4_of_a_kind":0,"full_house":0,"flush":0,"straight":0,"3_of_a_kind":0,"2_pair":0,"pair":0,"high":0} 
    for i in range(0,num_hands):
        deck = Deck() # have to run this each time so the deck doesnt run out of cards
        deck.shuffle()
        hand = deck.get_cards(5)
        suits,ranks = [card.suit for card in hand],[card.rank for card in hand]
        set_lens = [len(set(suits)),len(set(ranks))]
        if set_lens[0] == 1 and min(ranks) == max(ranks)-4: #a hand like Q,K,A,2,3 does not count as a straight
            types_of_hands["straight_flush"] += 1
        elif set_lens[1] == 2:
            dupes = [ranks.count(rank) for rank in set(ranks)]
            dupes.sort()
            if dupes == [2,3]:
                types_of_hands["full_house"] += 1
            elif dupes == [1,4]:
                types_of_hands["4_of_a_kind"] += 1
        elif set_lens[0] == 1:
            types_of_hands["flush"] += 1
        elif min(ranks) == max(ranks)-4 and set_lens[1] == 5:
            types_of_hands["straight"] += 1
        elif set_lens[1] == 3:
            dupes = [ranks.count(rank) for rank in set(ranks)]
            dupes.sort()
            if dupes == [1,2,2]:
                types_of_hands["2_pair"] += 1
            elif dupes == [1,1,3]:
                types_of_hands["3_of_a_kind"] += 1
        elif set_lens[1] == 4:
            types_of_hands["pair"] += 1
        else:
            types_of_hands["high"] += 1       
    return types_of_hands

In [171]:
monte_carlo(100000) # will take a few seconds

{'straight_flush': 2,
 '4_of_a_kind': 28,
 'full_house': 138,
 'flush': 186,
 'straight': 342,
 '3_of_a_kind': 2084,
 '2_pair': 4642,
 'pair': 42349,
 'high': 50229}

## Exercise 3: Tic-Tac-Toe

Revisit the tic-tac-toe game you developed in the functions exercises but now write it as a class with methods to do each of the main steps.  

In [172]:
class TicTacToe:
    
    def __init__(self):
        self.board = """
         {s1:^3} | {s2:^3} | {s3:^3}
        -----+-----+-----
         {s4:^3} | {s5:^3} | {s6:^3}
        -----+-----+-----      123
         {s7:^3} | {s8:^3} | {s9:^3}       456
                               789  
        """
        self.play = {}
        
    def initialize_board(self):
        for n in range(9):
            self.play["s{}".format(n+1)] = ""
            
    def show_board(self):
        """ display the playing board.  We take a dictionary with the current state of the board
        We rely on the board string to be a global variable"""
        print(self.board.format(**self.play))
        
    def get_move(self, n, xo):
        """ ask the current player, n, to make a move -- make sure the square was not 
            already played.  xo is a string of the character (x or o) we will place in
            the desired square """
        valid_move = False
        while not valid_move:
            idx = input("player {}, enter your move (1-9)".format(n))
            if self.play["s{}".format(idx)] == "":
                valid_move = True
            else:
                print("invalid: {}".format(self.play["s{}".format(idx)]))
        self.play["s{}".format(idx)] = xo
        
    def play_game(self):
        """ play a game of tic-tac-toe """
        self.initialize_board()
        self.show_board()
        game_over = False
        turn = 1 
        while not game_over:
            player = 2 - (turn % 2)
            self.get_move(player,['x','o'][player-1])
            self.show_board()
            if player == 1:
                check = [int(sq[1]) for sq in self.play if self.play[sq] == 'x']
            elif player == 2:
                check = [int(sq[1]) for sq in self.play if self.play[sq] == 'o']
            for i,j,k in zip([1,4,7,1,2,3,1,3],[2,5,8,4,5,6,5,5],[3,6,9,7,8,9,9,7]):
                if i in check and j in check and k in check and not game_over:
                    game_over = True
                    print("Congradulations, player {} has won!".format(player))     
            if turn == 9 and not game_over:
                print("Game over!  It's a tie.")
                game_over = True
            turn += 1

In [ ]:
game = TicTacToe()
game.play_game()